#**Art Painting 이미지를 사용한 정확도 올리기**

#**<문제>**
- 제공된 훈련데이터를 사용하여 모델을 훈련한 후 테스트 데이터에 대한 예측 카테고리 값을 csv파일로 제출하세요.
- 아래의 훈련데이터만을 사용하여 각 카테고리별로 분류가 잘 되도록 신경망을 구성하세요.
- 각 카테고리 별 정답 값은 아래와 같습니다.
> 'dog' : 0  
 'elephant' : 1  
 'giraffe' : 2  
 'guitar' : 3  
 'horse' : 4  
 'house' : 5  
 'person' : 6

#**<목적>**
주어진 데이터를 활용하여 정확도를 최대로 높이세요.

#**<훈련/테스트 데이터 세트 설명>**
훈련/테스트 데이터는 `Step 2. 데이터 다운로드`에서 확인할 수 있습니다.

- 테스트 입력 데이터 파일 : test.zip
- 훈련 데이터 파일: train.zip
- 예시 출력 데이터 파일: test_answer(sample).csv

###**데이터 설명**
- 데이터 셋은 art painting으로 구성되어 있으며 7개의 카테고리로 구분되어 있습니다.
- 7개의 카테고리로는 classes에서 정의한 dog, elephant, giraffe, guitar, horse, house, person를 포함합니다.
- 하나의 이미지의 사이즈는 [3(Channel) * 227(Width) * 227(Height)]이며 각 카테고리 별 예시 이미지는 아래에서 확인할 수 있습니다.
- 훈련 데이터는 art painting 1698장이며 테스트 데이터는 art painting 350장입니다.
- 아래의 이미지는 순서대로 dog, elephant, giraffe, guitar, horse, house, person에 대한 예시입니다.
```
classes = ['dog', 'elephant', 'giraffe','guitar','horse','house','person']
```

#**<최종 제출 파일>**#
- **submission.csv**  
    1.최종 제출 파일은 label 필드를 가지는 csv파일이며 테스트 데이터의 label을 값으로 가져야 합니다.  
    2.최종 제출 파일은 test_answer(sample).csv 파일과 같은 크기여야 하고 label값을 채워야합니다.  

#**<데이터 관련 안내>**#
- `train` 데이터 안에는 각 라벨별로 폴더가 나뉘어져 있으며 각 폴더 안의 파일을 사용하여 학습하여야 합니다.
- `test` 데이터는 라벨별로 폴더가 나뉘어져있지 않으며 임의의 순서대로 이미지가 섞여있습니다.
- 최종 제출물의 경우 데이터의 순서가 오름차순으로 정렬되어 있습니다. 이 순서대로 각 이미지에 대한 라벨 값을 작성하여 제출하세요.

#**<채점 방식>**#
테스트 데이터에서 추정한 label 값을 사용해 정확도를 구하고 이를 기준으로 채점합니다.

In [27]:
import os
import math
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [28]:
torch.manual_seed(17)

In [29]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset, DataLoader 정의

In [33]:
project_path = "/content/drive/MyDrive/Colab Notebooks/미술 작품 분류하기"

train_path = os.path.join(project_path, 'train')
test_path = os.path.join(project_path, 'test')

In [39]:
transform = transforms.Compose([transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_dataset = datasets.ImageFolder(train_path, transform=transform)
test_dataset = datasets.ImageFolder(test_path, transform=transform)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=16)

# Efiicientnet 모델

In [37]:
model = torchvision.models.efficientnet_b0(pretrained=True)
model.fc = torch.nn.Linear(in_features=512, out_features=7, bias=True)
torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

tensor([-0.0035,  0.0017,  0.0052,  0.0140, -0.0081, -0.0114, -0.0310])

In [43]:
from tqdm.notebook import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is using!")

model.to(device)
LEARNING_RATE = 0.0001
NUM_EPOCH = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_dataloader,
    "test" : test_dataloader
}

cuda:0 is using!


In [44]:
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      model.train()
    elif phase == "test":
      model.eval()

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      with torch.set_grad_enabled(phase == "train"):
        logits = model(images)
        _, preds = torch.max(logits, 1)  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward()
          optimizer.step()

      running_loss += loss.item() * images.size(0)
      running_acc += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc:
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss:
      best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

  0%|          | 0/107 [00:00<?, ?it/s]

현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.815, 평균 Accuracy : 0.821


  0%|          | 0/22 [00:00<?, ?it/s]

현재 epoch-0의 test-데이터 셋에서 평균 Loss : 7.232, 평균 Accuracy : 0.146


  0%|          | 0/107 [00:00<?, ?it/s]

...

현재 epoch-8의 test-데이터 셋에서 평균 Loss : 8.094, 평균 Accuracy : 0.166


  0%|          | 0/107 [00:00<?, ?it/s]

현재 epoch-9의 train-데이터 셋에서 평균 Loss : 0.038, 평균 Accuracy : 0.987


  0%|          | 0/22 [00:00<?, ?it/s]

현재 epoch-9의 test-데이터 셋에서 평균 Loss : 9.899, 평균 Accuracy : 0.123
학습 종료!
최고 accuracy : 0.16571427881717682, 최고 낮은 loss : 7.218498224530901
